
Pole In Motion 

Author - Amir Silberstein

In [1]:
%matplotlib -- inline
from math import *
import matplotlib.pyplot as plt
import numpy as np
from sympy.solvers import solve
from sympy import Symbol
import scipy.integrate

global g
g = 9.8 # m/s^2
k = 500

In [2]:
def atr(a): #angle to radian
    return (a*pi/180)

In [3]:

def Euler(f,t,y,dt):

    return y+f(y,t)*dt


def BackEuler(f,t,y,dt):

    tmpy=y+f(y,t)*dt
    return y+f(tmpy,t)*dt


def rk2(f,t,y,dt):

    k1 = f(t,y) * dt
    k2 = f(t + dt,y + k1) * dt
    return (y + 0.5*(k1 + k2))


def rk3(f,t,y,dt):

    k1 = f(t,y) * dt
    k2 = f(t + 0.5*dt,y + 0.5*k1) * dt
    k3 = f(t + 0.75*dt,y + 0.75*k2) * dt
    return y + (1/9)*(2*k1 + 3*k2 + 4*k3)


def rk4(f,t,y,dt):

    k1 = f(t,y) * dt
    k2 = f(t + 0.5*dt,y + 0.5*k1) * dt
    k3 = f(t + 0.5*dt,y + 0.5*k2) * dt
    k4 = f(t + dt,y + k3) * dt
    return y + (1/6)*(k1 + 2*k2 + 2*k3 + k4)


In [4]:
def hit(data):
    arrp = get_points(data) #array of the pole's vertex x,y [[x,y], [x,y], [x,y], [x,y]]
    for j in range(0, 3):
        if (arrp[j][1] <= 0 and min(arrp[:][1]) == arrp[j][1]):
            return [True, arrp[j]]
    return [False]

In [5]:
def deriv(t,data):
    #data = np.arr([x,y,vx,xy,omega,theta])
    #print(t,data, get_points(data, dim))
    x = data[0]
    y = data[1]
    vx = data[2]
    vy = data[3]
    omega = data[4]
    theta = data[5]
    dvx = 0
    hit1 = hit(data)
    if hit1[0] == True:
        hitp = hit1[1] #point that hit the ground [x,y]
        dw = (data[0]-hitp[0])*k*abs(hitp[1]) / moi
        dvy =  (k*abs(hitp[1])-m*g)/m
    else:
        dw = 0
        dvy = -g
    return np.array([vx,vy,dvx,dvy,dw,omega])

In [6]:
'''
returns arr of x,y points of the poles vertex.
'''

def get_points(data):

    r = sqrt((dim[0]/2)**2 + (dim[1]/2)**2) #distance between the middle of the pole and the vertex
    angle1 = atan(dim[1]/dim[0]) #angle between the hypotenuse

    if (data[5]%pi == 0):
        print (data[0])
        p1x = data[0]-(dim[0]/2)
        p2x = data[0]-(dim[0]/2)
        p3x = data[0]+(dim[0]/2)
        p4x = data[0]+(dim[0]/2)
        #
        p1y = data[1]+(dim[1]/2)
        p3y = data[1]+(dim[1]/2)
        p2y = data[1]-(dim[1]/2)
        p4y = data[1]-(dim[1]/2)
    elif (data[5]%pi == 0.5*pi):
        p2x = data[0]-dim[1]/2
        p4x = p2x
        p1x = data[0]+dim[1]/2
        p3x = p1x
        #
        p1y = data[1]+dim[0]/2
        p2y = p1y
        p3y = data[1]-dim[0]/2
        p4y = p3y
        #
    elif (data[5]%pi < 0.5*pi):
        angle1 = angle1
        p1x = data[0]-r*sin(data[5]-angle1)
        p2x = data[0]-r*cos(data[5]-angle1)
        p3x = data[0]+r*sin(data[5]+angle1)
        p4x = data[0]+r*cos(data[5]+angle1)
        #
        p1y = data[1]+r*cos(data[5]-angle1)
        p2y = data[1]+r*sin(data[5]-angle1)
        p3y = data[1]-r*cos(data[5]+angle1)
        p4y = data[1]-r*sin(data[5]+angle1)
        #
    elif (data[5]%pi > 0.5*pi):
        p1x = data[0]-r*sin(data[5]+angle1)
        p2x = data[0]-r*cos(data[5]+angle1)
        p3x = data[0]+r*sin(data[5]-angle1)
        p4x = data[0]+r*cos(data[5]-angle1)
        #
        p1y = data[1]-r*cos(data[5]+angle1)
        p2y = data[1]-r*sin(data[5]+angle1)
        p3y = data[1]+r*cos(data[5]-angle1)
        p4y = data[1]+r*sin(data[5]-angle1)

    return np.array([[p1x,p1y],[p2x,p2y],[p3x,p3y],[p4x,p4y]])

In [7]:
def total_energy(data):
    tot = 0.5*m*(data[3])**2 + 0.5*(data[4])**2*moi + abs(m*g*data[1])
    return tot
    

In [8]:
def start(f,t0 ,data0, dimensions, dt, n ,mass = 1):
    global m, dim, moi
    m = mass    
    dim = dimensions
    moi = m*(dim[0]**2+dim[1]**2)/12 # moment of inertia
    t = np.zeros(n)
    data = np.zeros([n,len(data0)])
    points = np.zeros([n, 4, 2])
    t_energy = np.zeros(n)
    t[0] = t0
    data[0] = data0
    points[0] = get_points(data[0])
    t_energy[0] = total_energy(data[0])
    
    for i in range(1,n):
        t[i] = t[i-1] + dt
        #ht = solve(data[i-1][1]+data[i-1][3]*ht+0.5*(-g)*ht**2, ht)[1]
        if (data[i-1][1]<=dim[0]/2):
            #print("ground")
            data[i] = scipy.integrate.solve_ivp(f,(t[i-1],t[i]),data[i-1]).y[:,-1]
            data[i][5] = data[i][5]%360
        else:
            #print("air")
            data[i] = [data[i-1][0]+data[i-1][2]*dt, data[i-1][1]+data[i-1][3]*dt, data[i-1][2], data[i-1][3]-g*dt, data[i-1][4], data[i-1][5]+data[i-1][5]*data[i-1][4]]
            data[i][5] = data[i][5]%360
        points[i] = get_points(data[i])
        t_energy[i] = total_energy(data[i])
        
        
    return t,data, points,t_energy

In [9]:
def make_graph(t ,data , points ,t_energy, g_type):
    '''
    g_type is the graph type
    g_type = 1 creates y(t) graph
    g_type = 2 creates vy(t) graph
    g_type = 3 creates w(t) graph
    g_type = 4 creates Et(t) graph
    '''
    x,y,vx,vy,omega,theta = zip(*data)
    
    if(g_type == 1):
        plt.plot(t,y)
        plt.grid()
        plt.xlabel("Time(s)")
        plt.ylabel("Position(y)(m)")
        plt.title("y(t)")
    if(g_type == 2):
        plt.plot(t,vy)
        plt.grid()
        plt.xlabel("Time(s)")
        plt.ylabel("Velocity(m/s)")
        plt.title("vy(t)")
    if(g_type == 3):
        plt.plot(t,omega)
        plt.grid()
        plt.xlabel("Time(s)")
        plt.ylabel("Omega(rad/s)")
        plt.title("w(t)")
    if(g_type == 4):
        plt.plot(t,t_energy)
        plt.grid()
        plt.xlabel("time(s)")
        plt.ylabel("Total Energy(J)")
        plt.title("Et(t)")
    if(g_type == 5):
        plt.plot(t,theta)
        plt.grid()
        plt.xlabel("time(s)")
        plt.ylabel("Theta(rad)")
        plt.title("Theta(t)")
    plt.show()

In [10]:
data0 = np.array([0,1,0,0, 0.5 ,atr(90)])
dimensions = [0.06 ,0.05]
mass = 0.05
dt = 10E-6
tmax = 1.25
t0 = 0
n = int(tmax / dt)

In [11]:
t,data,points,t_energy= start(deriv ,t0 ,data0, dimensions, dt, n ,mass)

In [12]:
for graph_type in range(1,5):
    make_graph(t, data, points, t_energy, graph_type)

#### '''
f = 
dim = dimensions of the pole = [length,width]
t = time
data = np.arr([x,y,vx,xy,omega,theta])
p = x,y (location) of the mid of the pole.
v = vx, vy velocity of the mid of the pole.

'''